In [2]:
using Gurobi
using CSV
using DataFrames
using JuMP

┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\Gurobi\do9v6.ji for Gurobi [2e9cd046-0924-5485-92f1-d5272153d98b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\CSV\HHBkp.ji for CSV [336ed68f-0bac-5ca0-87d4-7b16caf5d00b]
└ @ Base loading.jl:1190
┌ Info: Recompiling stale cache file C:\Users\steph\.julia\compiled\v1.0\JuMP\DmXqY.ji for JuMP [4076af6c-e467-56ae-b986-b466b2749572]
└ @ Base loading.jl:1190


In [10]:
# import Pkg
# Pkg.activate(@__DIR__)
# Pkg.instantiate()

  Updating registry at `C:\Users\steph\.julia\registries\General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`
 Resolving package versions...


In [3]:
df = CSV.read("ev_constraints.csv", header = true, missingstring="\n")

,station_1,station_2,station_3,station_4,station_5,station_6,station_7,station_8,station_9
,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰,Int64⍰
1,16,18,26,31,43,40,39,34,35
2,16,18,26,31,43,40,39,34,35
3,2,6,missing,missing,missing,missing,missing,missing,missing
4,6,missing,missing,missing,missing,missing,missing,missing,missing
5,3,4,5,19,missing,missing,missing,missing,missing
6,4,5,19,missing,missing,missing,missing,missing,missing
7,5,19,missing,missing,missing,missing,missing,missing,missing
8,3,4,5,19,missing,missing,missing,missing,missing
9,4,5,19,missing,missing,missing,missing,missing,missing


In [5]:
num_nodes = 50

m = Model(solver=GurobiSolver(OutputFlag=0))

@variable(m, x[1:num_nodes],Bin)

for row in eachrow(df)
    v = vec(convert(Array, row)) # no transpose necessary
    @constraint(m, sum(x[i] for i in v if !ismissing(i)) >= 1)
end

@objective(m, Min, sum(x[i] for i in 1:num_nodes))


ErrorException: The solver= keyword is no longer available in JuMP 0.19 and later. See the JuMP documentation (http://www.juliaopt.org/JuMP.jl/latest/) for latest syntax.

In [ ]:
# Additional constraints

# Safety concerns
# Assume some array of blacklisted notes, b
@constraint(m, sum(x[i] for i in b) == 0)

# Environmental cost or electricity cost
# Assume some environmental or electicity cost array, e,
# where e[i] is the environmental or electricity cost associated
# with installing an EV charger at node i
@objective(m, Min, sum(x[i]*e[i] for i in 1:num_nodes))

In [71]:
status = solve(m)

Academic license - for non-commercial use only


:Optimal

In [1]:
for row = 1:length(getvalue(x))
    if getvalue(x)[row] == 1.0
        println(row)
    end
end

UndefVarError: UndefVarError: getvalue not defined

In [72]:
getvalue(x)

18-element Array{Float64,1}:
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0
 0.0
 1.0
 1.0
 1.0
 1.0
 1.0
 0.0
 1.0
 1.0
 0.0
 1.0
 0.0

,station_1,station_2,station_3
1,2,missing,missing


In [64]:
 ismissing(df[1,1])

false

In [68]:
m

Feasibility problem with:
 * 114 linear constraints
 * 18 variables: 18 binary
Solver is Gurobi